In [4]:
!pip install pandas
!pip install transformers
!pip install biopython
!pip install vllm
!pip install numpy


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 155.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 132.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 145.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 44.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec

In [1]:
from __future__ import annotations
import pandas as pd
from transformers import set_seed
import json
from Bio import Entrez
import vllm
import argparse
from tqdm import tqdm
import numpy as np
import os
import time

In [2]:
model = vllm.LLM(model="lexu14/porpoise1", max_model_len=4000)

sampling_config = vllm.SamplingParams(
    temperature=0,
    top_k=20,
    top_p=0.95,
    max_tokens=1,
)

INFO 07-24 20:11:16 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='lexu14/porpoise1', speculative_config=None, tokenizer='lexu14/porpoise1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=lexu14/porpoise1, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 07-24 20:11:16 selector.py:170] Cannot use FlashAttention-2 backend due to sliding window.
INFO 07-24 20:11:16 selector.py:54] Using XFormers backend.


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 07-24 20:11:21 model_runner.py:692] Loading model weights took 7.1183 GB
INFO 07-24 20:11:22 gpu_executor.py:102] # GPU blocks: 5521, # CPU blocks: 682
INFO 07-24 20:11:25 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-24 20:11:25 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-24 20:11:35 model_runner.py:1181] Graph capturing finished in 10 secs.


In [10]:
abstract = "Type 2 diabetes mellitus (DM) is characterized by insulin resistance and pancreatic beta cell dysfunction. In high-risk subjects, the earliest detectable abnormality is insulin resistance in skeletal muscle. Impaired insulin-mediated signaling, gene expression, glycogen synthesis, and accumulation of intramyocellular triglycerides have all been linked with insulin resistance, but no specific defect responsible for insulin resistance and DM has been identified in humans. To identify genes potentially important in the pathogenesis of DM, we analyzed gene expression in skeletal muscle from healthy metabolically characterized nondiabetic (family history negative and positive for DM) and diabetic Mexican-American subjects. We demonstrate that insulin resistance and DM associate with reduced expression of multiple nuclear respiratory factor-1 (NRF-1)-dependent genes encoding key enzymes in oxidative metabolism and mitochondrial function. Although NRF-1 expression is decreased only in diabetic subjects, expression of both PPAR gamma coactivator 1-alpha and-beta (PGC1-alpha/PPARGC1 and PGC1-beta/PERC), coactivators of NRF-1 and PPAR gamma-dependent transcription, is decreased in both diabetic subjects and family history-positive nondiabetic subjects. Decreased PGC1 expression may be responsible for decreased expression of NRF-dependent genes, leading to the metabolic disturbances characteristic of insulin resistance and DM."
hyp = "There exists an interaction between diabetes and PPAR."

In [19]:
data = [f"Abstract: {abstract}\nHypothesis: {hyp}\nInstructions: Classify this abstract as either 0 (Not Relevant) or 1 (Relevant) for evaluating the provided hypothesis.\nScore: "] * 10000

In [20]:
import time
start_time = time.time()
generated = model.generate(data, sampling_config)
print(time.time() - start_time)

Processed prompts:  48%|████▊     | 4834/10000 [02:48<02:58, 28.87it/s, est. speed input: 11945.58 toks/s, output: 28.65 toks/s]

KeyboardInterrupt: 

In [6]:
generated[0].outputs[0].text

'1'